# ESM Folding

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import py3Dmol
from proteome import protein
from proteome.models.folding.esm.modeling import ESMForFolding

In [3]:
sequence = 'MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH'

In [10]:
folder = ESMForFolding(chunk_size=512)
predicted_protein, mean_plddt = folder.fold(sequence)

In [11]:
result_pdb = protein.to_pdb(predicted_protein)

In [12]:
with open("esmfold_3B_v0_folding.pdb", mode="w") as f:
    f.writelines(result_pdb)

In [32]:
import numpy as np
from proteome.models.folding.openfold.np.relax.utils import overwrite_b_factors

In [33]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]

banded_b_factors = []
for plddt in predicted_protein.b_factors.mean(axis=-1):
    for idx, (min_val, max_val, _) in enumerate(PLDDT_BANDS):
        if plddt >= min_val and plddt <= max_val:
            banded_b_factors.append(idx)
            break
        
banded_b_factors = np.array(banded_b_factors)[:, None] * predicted_protein.atom_mask
to_visualize_pdb = overwrite_b_factors(result_pdb, banded_b_factors)

In [35]:
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(to_visualize_pdb)
color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol